# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [ ]:
#Softmax는 Score점수에 확률이라는 의미를 부여함으로 분류를 더 잘할 수 있도록 하는 Loss함수를 새로 만듬.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!wget -P ./drive/My\ Drive http://cs231n.stanford.edu/assignments/2017/spring1617_assignment1.zip 

--2020-11-14 09:06:20--  http://cs231n.stanford.edu/assignments/2017/spring1617_assignment1.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43035 (42K) [application/zip]
Saving to: ‘./drive/My Drive/spring1617_assignment1.zip.6’

spring1617_assignme 100%[===================>]  42.03K  --.-KB/s    in 0.1s    

2020-11-14 09:06:21 (310 KB/s) - ‘./drive/My Drive/spring1617_assignment1.zip.6’ saved [43035/43035]



In [3]:
!mkdir ./drive/My\ Drive/assignment1

mkdir: cannot create directory ‘./drive/My Drive/assignment1’: File exists


In [4]:
!unzip ./drive/My\ Drive/spring1617_assignment1.zip -d ./drive/My\ Drive/assignment1

Archive:  ./drive/My Drive/spring1617_assignment1.zip
replace ./drive/My Drive/assignment1/assignment1/two_layer_net.ipynb? [y]es, [n]o, [A]ll, [N]one, [r]ename: All
  inflating: ./drive/My Drive/assignment1/assignment1/two_layer_net.ipynb  
  inflating: ./drive/My Drive/assignment1/assignment1/README.md  
  inflating: ./drive/My Drive/assignment1/assignment1/collectSubmission.sh  
  inflating: ./drive/My Drive/assignment1/assignment1/svm.ipynb  
  inflating: ./drive/My Drive/assignment1/assignment1/requirements.txt  
  inflating: ./drive/My Drive/assignment1/assignment1/setup_googlecloud.sh  
  inflating: ./drive/My Drive/assignment1/assignment1/start_ipython_osx.sh  
  inflating: ./drive/My Drive/assignment1/assignment1/knn.ipynb  
  inflating: ./drive/My Drive/assignment1/assignment1/frameworkpython  
  inflating: ./drive/My Drive/assignment1/assignment1/cs231n/gradient_check.py  
  inflating: ./drive/My Drive/assignment1/assignment1/cs231n/datasets/get_datasets.sh  
  inflating: ./

In [5]:
%cd /content/drive/My Drive/assignment1/assignment1

/content/drive/My Drive/assignment1/assignment1


In [6]:
!ls

collectSubmission.sh  k_nearest_neighbor.py   setup_googlecloud.sh
cs231n		      k_nearest_neighbor.pyc  softmax.ipynb
drive		      knn.ipynb		      start_ipython_osx.sh
features.ipynb	      README.md		      svm.ipynb
frameworkpython       requirements.txt	      two_layer_net.ipynb


In [7]:
%cd /content/drive/My Drive/assignment1/assignment1/cs231n/datasets
!./get_datasets.sh

/content/drive/My Drive/assignment1/assignment1/cs231n/datasets
--2020-11-14 09:06:40--  http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  61.1MB/s    in 2.7s    

2020-11-14 09:06:43 (61.1 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [8]:
!ls ./cifar-10-batches-py

batches.meta  data_batch_2  data_batch_4  readme.html
data_batch_1  data_batch_3  data_batch_5  test_batch


In [9]:
%cd /content/drive/My Drive/assignment1/assignment1
!ls

/content/drive/My Drive/assignment1/assignment1
collectSubmission.sh  k_nearest_neighbor.py   setup_googlecloud.sh
cs231n		      k_nearest_neighbor.pyc  softmax.ipynb
drive		      knn.ipynb		      start_ipython_osx.sh
features.ipynb	      README.md		      svm.ipynb
frameworkpython       requirements.txt	      two_layer_net.ipynb


In [10]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [11]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [14]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.364694
sanity check: 2.302585


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** *Fill this in*


In [15]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 2.259909 analytic: 2.259909, relative error: 3.407491e-08
numerical: 1.119288 analytic: 1.119288, relative error: 4.614104e-08
numerical: -3.848080 analytic: -3.848081, relative error: 1.777664e-08
numerical: 0.774704 analytic: 0.774704, relative error: 3.071544e-08
numerical: 1.356267 analytic: 1.356267, relative error: 1.119543e-08
numerical: 1.673897 analytic: 1.673897, relative error: 9.767969e-09
numerical: 0.225261 analytic: 0.225261, relative error: 2.202128e-07
numerical: 3.092594 analytic: 3.092594, relative error: 3.631454e-08
numerical: -0.109030 analytic: -0.109030, relative error: 3.317889e-07
numerical: -0.916715 analytic: -0.916715, relative error: 1.484548e-08
numerical: -3.290545 analytic: -3.290546, relative error: 6.984473e-09
numerical: 0.825297 analytic: 0.825297, relative error: 2.468576e-08
numerical: 0.575016 analytic: 0.575016, relative error: 6.506854e-08
numerical: -0.106774 analytic: -0.106774, relative error: 2.586638e-07
numerical: 1.332694 anal

In [16]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.364694e+00 computed in 18.451733s
vectorized loss: 2.364694e+00 computed in 0.015714s
Loss difference: 0.000000
Gradient difference: 0.000000


In [27]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for LR in learning_rates:
    for REG in regularization_strengths:
        model = Softmax()
        loss_hist = model.train(X_train, y_train, learning_rate=LR, reg=REG,num_iters=1500, verbose=False)
        train_accuracy = np.mean(y_train == model.predict(X_train))
        val_accuracy = np.mean(y_val == model.predict(X_val))
        results[(LR, REG)] = (train_accuracy, val_accuracy)
        if val_accuracy > best_val:
            best_val = val_accuracy
            best_model = model
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

AttributeError: ignored

In [28]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

AttributeError: ignored

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])